In [1]:
import pickle as pk
import pandas as pd
import numpy as np
import datetime
import matplotlib.dates as dates

In [2]:
with open('/home/jovyan/data/botpt/2019bottom_pressure15s_F.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptF = pd.DataFrame(botpt_data)
df_botptF['bottom_pressure'] = df_botptF['bottom_pressure'].astype(float)
df_botptF['depth']=df_botptF['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptF.index.to_pydatetime()]
df_botptF['epoch'] = epoch
df_botptF= df_botptF.sort_index()
df_botptF.index.name= 'Date'
del df_botptF['epoch']
del df_botptF['bottom_pressure']
df_botptF.tail()

,depth
Date,
2019-06-27 23:58:45,1510.264895
2019-06-27 23:59:00,1510.266367
2019-06-27 23:59:15,1510.267839
2019-06-27 23:59:30,1510.268494
2019-06-27 23:59:45,1510.269148


In [3]:
df_botptF.loc['2018-02-17']

,depth
Date,
2018-02-17 00:00:00,1510.675793
2018-02-17 00:00:15,1510.673176
2018-02-17 00:00:30,1510.669414
2018-02-17 00:00:45,1510.665815
2018-02-17 00:01:00,1510.662053
...,...
2018-02-17 23:58:45,1511.015210
2018-02-17 23:59:00,1511.012920
2018-02-17 23:59:15,1511.011121


In [4]:
with open('/home/jovyan/data/botpt/2019bottom_pressure15s_E.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE= df_botptE.sort_index()
df_botptE.index.name= 'Date'
del df_botptE['epoch']
del df_botptE['bottom_pressure']
df_botptE.tail()

,depth
Date,
2019-06-27 23:58:45,1501.491919
2019-06-27 23:59:00,1501.494209
2019-06-27 23:59:15,1501.495027
2019-06-27 23:59:30,1501.495518
2019-06-27 23:59:45,1501.496826


#### Merge BOTPT E and BOTPT F 

In [5]:
test = pd.merge(df_botptF, df_botptE,how='outer', indicator=True, left_index=True, right_index=True, suffixes=('_F', '_E'))

In [6]:
df_botptMerge = test[test['_merge'] == 'both']
del df_botptMerge['_merge']

In [7]:
df_botptMerge = df_botptMerge.loc['2017-1-1 00:00:00':'2017-1-10 00:00:00']

In [8]:
df_botptMerge.tail()

,depth_F,depth_E
Date,,
2017-01-09 18:29:45,1512.271460,1503.054707
2017-01-09 18:30:00,1512.270806,1503.052253
2017-01-09 18:30:15,1512.268352,1503.050781
2017-01-09 18:30:30,1512.265735,1503.050618
2017-01-09 18:30:45,1512.264590,1503.049963


#### Calculate Depth difference 

In [9]:
depthDiff = df_botptMerge['depth_E'].values - df_botptMerge['depth_F'].values

In [28]:
depthDiff = df_botptMerge['diff'].abs()

In [30]:
# df_botptMerge['diff'] = depthDiff
# df_botptMerge['diff'] = abs(depthDiff)

In [29]:
depthDiff

Date
2017-01-01 00:00:00    9.224441
2017-01-01 00:00:15    9.223459
2017-01-01 00:00:30    9.224932
2017-01-01 00:00:45    9.226567
2017-01-01 00:01:00    9.225586
                         ...   
2017-01-09 18:29:45    9.216753
2017-01-09 18:30:00    9.218552
2017-01-09 18:30:15    9.217571
2017-01-09 18:30:30    9.215117
2017-01-09 18:30:45    9.214626
Name: diff, Length: 50524, dtype: float64

In [33]:
df_botptMerge['diff'] = depthDiff

#### Create time and height vectors for plotting 

In [34]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_botptMerge['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMerge.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

#### Use Groupby to create one day mean measurements

In [35]:
df_botptMerge['date']=pd.DatetimeIndex(df_botptMerge.index).date
df_botptMerge

,depth_F,depth_E,diff,date
Date,,,,
2017-01-01 00:00:00,1511.468474,1502.244033,9.224441,2017-01-01
2017-01-01 00:00:15,1511.466184,1502.242725,9.223459,2017-01-01
2017-01-01 00:00:30,1511.464875,1502.239944,9.224932,2017-01-01
2017-01-01 00:00:45,1511.463894,1502.237327,9.226567,2017-01-01
2017-01-01 00:01:00,1511.460786,1502.235200,9.225586,2017-01-01
...,...,...,...,...
2017-01-09 18:29:45,1512.271460,1503.054707,9.216753,2017-01-09
2017-01-09 18:30:00,1512.270806,1503.052253,9.218552,2017-01-09
2017-01-09 18:30:15,1512.268352,1503.050781,9.217571,2017-01-09


In [14]:
df_botptMean=df_botptMerge.groupby('date').mean()
df_botptMean

,depth_F,depth_E,diff
date,,,
2017-01-01,1511.190229,1501.968277,-9.221951
2017-01-02,1511.150868,1501.930328,-9.220541
2017-01-03,1511.144084,1501.923073,-9.221010
2017-01-04,1511.154041,1501.934419,-9.219622
2017-01-05,1511.157324,1501.938949,-9.218375
2017-01-06,1511.154243,1501.934905,-9.219338
2017-01-07,1511.186865,1501.968706,-9.218159
2017-01-08,1511.203105,1501.985098,-9.218008
2017-01-09,1511.335411,1502.083856,-9.251555


In [65]:
df_botptMean = df_botptMean[(df_botptMean['date'] > '2017-1-1') & (df_botptMean['date'] <= '2017-1-11')]

KeyError: 'date'

#### Create time and height vectors for plotting 

In [66]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_botptMean['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [67]:
height

[-9.221951494428811,
 -9.220540553198909,
 -9.221010175916883,
 -9.219622066921684,
 -9.21837450112235,
 -9.219337828318297,
 -9.218159014807796,
 -9.218007509867345,
 -9.251554817041749,
 -9.217460988989377,
 -9.215046958499494,
 -9.213947688632528,
 -9.2134491848416,
 -9.212935800340446,
 -9.212929552714025,
 -9.211074178059883,
 -9.211686559041366,
 -9.209554329342316,
 -9.209729214082104,
 -9.209209971533843,
 -9.209388057284881,
 -9.206200376298689,
 -9.206468910641142,
 -9.207286980522989,
 -9.205540967729364,
 -9.204338356441896,
 -9.204579344431542,
 -9.204535071478974,
 -9.20355680677626,
 -9.203602064139014,
 -9.203164856339649,
 -9.203754629347056,
 -9.20391164355811,
 -9.204841460757796,
 -9.203830708397755,
 -9.207030742645257,
 -9.212843647850892,
 -9.211854871762254,
 -9.21112035369874,
 -9.211804383595783,
 -9.212734371185283,
 -9.210902851104722,
 -9.212309532589389,
 -9.211475957234729,
 -9.211205065833179,
 -9.212142618342899,
 -9.215475645065315,
 -9.21153746795656,

In [63]:
import pickle


with open('/jovyan/data/botpt/2019bottom_pressure15s_E.pkl, 'rb') as f:
    botpt_E = pickle.load(f)

SyntaxError: EOL while scanning string literal (<ipython-input-63-fb23c3416c47>, line 4)

In [64]:
botpt_E

,bottom_pressure
2017-02-08 06:00:00,2242.541992
2017-02-08 06:00:15,2242.541748
2017-02-08 06:00:30,2242.540771
2017-02-08 06:00:45,2242.541748
2017-02-08 06:01:00,2242.544922
...,...
2017-01-23 23:58:45,2240.078125
2017-01-23 23:59:00,2240.083496
2017-01-23 23:59:15,2240.083252
2017-01-23 23:59:30,2240.077637


In [7]:
with open('2019bottom_pressure15s_F.pkl', 'rb') as f:
    botpt_F = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '2019bottom_pressure15s_F.pkl'

In [12]:
botpt_F.iloc[1, 10]

IndexError: single positional indexer is out-of-bounds

In [11]:
botpt_F[1, 10]

KeyError: (1, 10)